In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import io

In [3]:
save_path = r'E:\My Drive\Nutrition5kDataset'
image_df = pd.read_pickle(os.path.join(save_path, 'dish_images.pkl'))
dishes = pd.read_excel(os.path.join(save_path, 'dishes.xlsx'))
dish_ingredients = pd.read_excel(os.path.join(save_path, 'dish_ingredients.xlsx'))
ingredients = pd.read_excel(os.path.join(save_path, 'ingredients.xlsx'))

In [4]:
image_df = pd.merge(image_df, dishes, left_on='dish', right_on='dish_id', how='left').drop('dish_id', axis=1)
image_df

,dish,rgb_image,depth_image,total_mass,total_calories,total_fat,total_carb,total_protein
0,dish_1559243887,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,289.050018,50,24.750000,11.350000,11.200001
1,dish_1563568319,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,604.323303,347,28.784986,53.148632,37.726025
2,dish_1558641007,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,23.369999,57,0.114000,5.700000,0.513000
3,dish_1558109945,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,74.360001,143,0.286000,0.429000,20.020000
4,dish_1563216412,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,172.449982,205,0.714000,36.610001,5.238000
...,...,...,...,...,...,...,...,...
3485,dish_1563564828,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,452.214478,355,25.004564,38.766243,34.570633
3486,dish_1566587689,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,196.846970,270,7.475343,15.383455,18.580095
3487,dish_1575307819,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,0.000000,114,0.000000,0.000000,0.000000
3488,dish_1559242155,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,116.116005,203,0.203000,30.855999,0.812000


In [5]:
# Split the data
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(image_df[image_df[['total_mass', 'total_fat', 'total_carb', 'total_protein']].sum(axis=1) > 0], test_size=0.2, random_state=42)
train_df.shape, val_df.shape

((2608, 8), (652, 8))

In [6]:
import torch
import torch_directml
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from PIL import Image
import io
from tqdm import tqdm

# ====== Image Transform ======
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ====== Dataset Class ======
class DishDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(io.BytesIO(row['rgb_image'])).convert("RGB")
        if self.transform:
            image = self.transform(image)
        targets = torch.tensor([
            row['total_calories'],
            row['total_fat'],
            row['total_carb'],
            row['total_protein']
        ], dtype=torch.float32)
        return image, targets

# ====== Set Device ======
device = torch_directml.device()
print(f"Using DirectML device: {device}")

# ====== Data Loaders ======
train_loader = DataLoader(DishDataset(train_df, transform), batch_size=8, shuffle=True)
val_loader = DataLoader(DishDataset(val_df, transform), batch_size=8, shuffle=False)

# ====== Model Definition ======
class NutrientPredictor(nn.Module):
    def __init__(self):
        super(NutrientPredictor, self).__init__()
        base_model = models.resnet18(pretrained=True)
        base_model.fc = nn.Identity()
        self.feature_extractor = base_model
        self.regressor = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.regressor(features)

# ====== Training & Evaluation Function ======
def train_model(model, train_loader, val_loader, epochs=5):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, targets = images.to(device), targets.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Train Loss: {total_loss / len(train_loader):.4f}")

    # ====== Evaluation ======
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            outputs = model(images)
            y_pred.append(outputs.cpu().numpy())
            y_true.append(targets.numpy())

    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')

    print("\nValidation Results:")
    print(f"MSE (Calories, Fat, Carbs, Protein): {mse}")
    print(f"R² Scores: {r2}")

    return model, y_true, y_pred

Using DirectML device: privateuseone:0


In [10]:
# ====== Initialize and Train ====== (on cpu)
model = NutrientPredictor()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model1, y_true1, y_pred1 = train_model(model, train_loader, val_loader, epochs=1)

Epoch 1/1: 100%|█████████████████████████████████████████████████████████████████████| 326/326 [02:28<00:00,  2.20it/s]


Epoch 1, Train Loss: 12051.7896

Validation Results:
MSE (Calories, Fat, Carbs, Protein): [28715.861     209.66827   269.6481    337.17294]
R² Scores: [-0.12808073 -0.03652835  0.05217797  0.21765244]


In [21]:
# ====== Initialize and Train ====== (on gpu)
model = NutrientPredictor()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model1, y_true1, y_pred1 = train_model(model, train_loader, val_loader, epochs=1)

C:\Users\ASUS\anaconda3\envs\jupyterlab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ASUS\anaconda3\envs\jupyterlab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/1: 100%|█████████████████████████████████████████████████████████████████████| 326/326 [05:22<00:00,  1.01it/s]


Epoch 1, Train Loss: 18584.6878

Validation Results:
MSE (Calories, Fat, Carbs, Protein): [72324.625    380.3474   684.0191   755.8154]
R² Scores: [-1.8412178 -0.8803077 -1.4043498 -0.7537302]
